<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Pratical-Keras-Tutorials/Keras_Custom_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

# Custom Layer with Subclassing API

In [2]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()

    w_init= tf.random_normal_initializer()
    self.w=tf.Variable(
        initial_value=w_init(shape=(input_dim,units), dtype="float32"),
        trainable=True
    )
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(units,), dtype="float32"),
        trainable=True
    )
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [3]:
x = tf.ones((2,2))
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [6]:
linear_layer = Linear(4,2)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.02182704, -0.06834333, -0.08846537, -0.10084669],
       [ 0.02182704, -0.06834333, -0.08846537, -0.10084669]],
      dtype=float32)>

# Custom Layer using add_weight

In [13]:
from tensorflow.python.ops.init_ops_v2 import Initializer

class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()
    self.w = self.add_weight(shape=(input_dim, units),
                             initializer="random_normal",
                             trainable = True)
    self.b = self.add_weight(shape=(units,),
                             initializer = "zeros",
                             trainable = True)
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [14]:
x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)

In [15]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.03019988, -0.0250623 , -0.14587434, -0.03216495],
       [ 0.03019988, -0.0250623 , -0.14587434, -0.03216495]],
      dtype=float32)>

# Custom Layer without the shape of the inputs

In [16]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32):
    super(Linear,self).__init__()
    self.units = units
  def build(self,input_shape):
    self.w = self.add_weight(shape=(input_shape[-1],self.units),
                             initializer = "random_normal",
                             trainable = True)
    self.b = self.add_weight(shape = (self.units,),
                             initializer = "random_normal",
                             trainable = True)
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [17]:
linear_layer = Linear(32)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[-0.06740423, -0.05197623,  0.04423926, -0.01153909, -0.00029627,
         0.08556522,  0.0124561 , -0.06567153, -0.08861082, -0.03360906,
         0.06100948, -0.11515134, -0.03720739, -0.05778199,  0.01721879,
         0.20487744,  0.07794537, -0.10588718, -0.01079212,  0.04534256,
        -0.07351579,  0.11206988,  0.14135364,  0.07469255,  0.05566546,
         0.01701791, -0.07348507,  0.03512447,  0.09966917,  0.13252887,
        -0.11443132, -0.04413957],
       [-0.06740423, -0.05197623,  0.04423926, -0.01153909, -0.00029627,
         0.08556522,  0.0124561 , -0.06567153, -0.08861082, -0.03360906,
         0.06100948, -0.11515134, -0.03720739, -0.05778199,  0.01721879,
         0.20487744,  0.07794537, -0.10588718, -0.01079212,  0.04534256,
        -0.07351579,  0.11206988,  0.14135364,  0.07469255,  0.05566546,
         0.01701791, -0.07348507,  0.03512447,  0.09966917,  0.13252887,
        -0.11443132, -0.04413957]], dtyp

# Layers are recursively composable

In [18]:
class MLPBlock(keras.layers.Layer):
  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)
  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [19]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3,64)))

In [20]:
mlp.weights

[<Variable path=mlp_block/linear_6/variable_6, shape=(64, 32), dtype=float32, value=[[ 0.02996054  0.01567549  0.07191271 ... -0.02443671  0.09260366
   -0.11078904]
  [-0.02201861 -0.02452221 -0.00194104 ... -0.04274567  0.04523487
   -0.07730837]
  [ 0.07420205  0.08802603  0.04777047 ...  0.02915013 -0.05667111
    0.015878  ]
  ...
  [-0.02311507 -0.02580372 -0.0811302  ...  0.00262618  0.030489
   -0.12262056]
  [ 0.06868153 -0.03306196 -0.04219622 ...  0.02389799 -0.05933542
    0.02211909]
  [ 0.0032062   0.01186916  0.06928699 ... -0.09748977  0.17630552
    0.04285609]]>,
 <Variable path=mlp_block/linear_6/variable_7, shape=(32,), dtype=float32, value=[-0.0184292   0.014743   -0.00450797  0.01619827 -0.06954794  0.00752611
   0.00936912 -0.05958122  0.00039945 -0.03188058  0.01472428 -0.02603077
  -0.05340106 -0.00231474  0.03335221 -0.02725818 -0.01818893  0.02875867
   0.05787683 -0.06295809  0.00912935  0.03541419  0.01161952  0.02203942
  -0.02222626  0.01662326 -0.0045120

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎